 # Universality Verification Experiment for Scaling Law (D-Scaling)

## Core Theory and Functionality
This version represents the final presentation of the theory. Building upon v12, it restores the classic performance scaling plot to form a complete narrative chain:

1.  **Restored Performance Plot (Law 1)**: The "Performance vs. Data Size" plot is reinstated as the first figure, intuitively demonstrating the direct connection of the experiment to Scaling Law research.
2.  **Complete Theoretical Narrative**: The final chart comprises 5 parts, holistically presenting the entire unified theory—from macroscopic performance, to the decomposition of internal components, and finally to the core predictive law.
3.  **Data Persistence**: The functionality to save experimental results to a CSV file has been retained.

In [ ]:
# Download the dataset on the first run
from torchvision import datasets, transforms
try:
    datasets.FashionMNIST('./data', train=True, download=True, transform=transforms.ToTensor())
    datasets.FashionMNIST('./data', train=False, download=True, transform=transforms.ToTensor())
    print("FashionMNIST dataset is ready.")
except Exception as e:
    print(f"Could not download dataset. Error: {e}")

In [ ]:
import sys
import os

# Add the experiment's directory to the Python path
workspace_path = "/D-Scaling" # Please adjust this path according to your setup
if workspace_path not in sys.path:
    sys.path.append(workspace_path)
    print(f"Added path: {workspace_path}")
else:
    print(f"Path {workspace_path} is already in the Python path")

# Confirm the existence of the logic module file
file_path = os.path.join(workspace_path, "MLP_D_logic.py") # Find and replace with the logic module for the desired model
if os.path.exists(file_path):
    print(f"✓ File exists: {file_path}")
else:
    print(f"✗ File not found: {file_path}")

# Attempt to import the module
try:
    import MLP_D_logic # Replace with the logic module for the desired model
    print("✓ Module imported successfully!")
    
    # Check for the presence of the necessary function
    if hasattr(MLP_D_logic, 'run_training_task'):
        print("✓ Found function: run_training_task")
        run_training_task = MLP_D_logic.run_training_task
    else:
        print("✗ Function not found: run_training_task")
        
except ImportError as e:
    print(f"✗ Import failed: {e}")
    print("Attempting alternative import method...")
    
    # Use importlib as a fallback
    try:
        import importlib.util
        spec = importlib.util.spec_from_file_location("MLP_D_logic", file_path)
        MLP_D_logic = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(MLP_D_logic)
        print("✓ Successfully imported using importlib")

        if hasattr(MLP_D_logic, 'run_training_task'):
            print("✓ Found function: run_training_task")
            run_training_task = MLP_D_logic.run_training_task
        else:
            print("✗ Function not found: run_training_task")
            
    except Exception as e:
        print(f"✗ Alternative import also failed: {e}")

In [ ]:
import numpy as np

# --- 1. Single Run Configuration ---
SINGLE_RUN_CONFIG = {
    "seed": 42,
    "epochs": 60,
}

# --- 2. Base Experiment Parameters ---
BASE_CONFIG = {
    "d_values": np.logspace(2, 4.7, 25).astype(int),
    "num_points_for_linf_estimation": 5,
    "batch_size": 128,
    "learning_rate": 0.001,
    "analysis_sample_size": 30,
    "w1": 0.5, 
    "w2": 0.5,
}
BASE_CONFIG['epochs'] = SINGLE_RUN_CONFIG['epochs']

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.stats import linregress
import torch
import os

from MLP_D_logic import run_training_task, TheoryAnalyzer # Replace with the logic module for the desired model

if __name__ == '__main__':
    seed = SINGLE_RUN_CONFIG['seed']
    d_values = BASE_CONFIG['d_values']
    tasks = [(seed, d, BASE_CONFIG, 0 if torch.cuda.is_available() else -1) for d in d_values]
    
    results = []
    for task_args in tqdm(tasks, desc=f"Running experiment for Seed {seed}, Epochs {BASE_CONFIG['epochs']}"):
        result = run_training_task(task_args)
        if result: results.append(result)
    
    if not results:
        print("No results generated.")
    else:
        df = pd.DataFrame(results).sort_values('data_size_d')
        
        plt.style.use('seaborn-v0_8-whitegrid')
        fig, axes = plt.subplots(1, 5, figsize=(40, 8))
        fig.suptitle(f'Cognitive Investment Model - Unified Theory (Seed={seed}, Epochs={BASE_CONFIG["epochs"]})', fontsize=24, y=0.98)
        
        # --- Fitting Functions ---
        def power_law_fit(x, y):
            mask = (y > 0) & (x > 0) & np.isfinite(y) & np.isfinite(x)
            if mask.sum() < 2: return 0, 1, 0, np.full_like(x, np.nan, dtype=float)
            lx, ly = np.log10(x[mask]), np.log10(y[mask])
            s, i, r, p, _ = linregress(lx, ly); r2 = r**2
            return r2, p, s, 10**(s*np.log10(x)+i)
            
        def exp_decay_fit(x, y):
            mask = (y > 0) & np.isfinite(y) & np.isfinite(x)
            if mask.sum() < 2: return 0, 1, 0, np.full_like(x, np.nan, dtype=float)
            lx, ly = x[mask], np.log(y[mask])
            s, i, r, p, _ = linregress(lx, ly); r2 = r**2
            return r2, p, -s, np.exp(s*x+i)

        # --- Data Preparation for Reducible Metrics ---
        n_est = BASE_CONFIG['num_points_for_linf_estimation']
        
        L_inf = df['final_test_loss'].tail(n_est).mean()
        df['reducible_loss'] = df['final_test_loss'] - L_inf
        hsie_inf = df['final_hsie'].tail(n_est).mean()
        df['reducible_hsie'] = df['final_hsie'] - hsie_inf
        
        htse_0 = df['final_htse'].head(n_est).mean()
        df['reducible_htse'] = df['final_htse'] - htse_0

        x_d = df['data_size_d'].values
        
        # --- [Plot 1 - Restored] Performance Scaling Law ---
        ax0 = axes[0]
        r2_1, p_1, s_1, fit_1 = power_law_fit(x_d, df['reducible_loss'])
        ax0.set_title('Law 1: Performance vs. Data (Power Law)')
        ax0.plot(x_d, df['final_test_loss'], 'o', color='blue')
        ax0.plot(x_d, fit_1 + L_inf, '--', color='red')
        ax0.text(0.95, 0.95, f'$R^2={r2_1:.2f}, p={p_1:.1e}$\n$L-L_\infty \propto D^{{{s_1:.2f}}}$', ha='right', va='top', transform=ax0.transAxes, bbox=dict(fc='wheat', alpha=0.5))
        ax0.set_ylabel('Final Test Loss')

        # --- [Plot 2] Component 1: Abstraction ---
        r2_2, p_2, s_2, fit_2 = power_law_fit(x_d, df['reducible_htse'])
        axes[1].set_title('Component 1: Abstraction (Power Law)');
        axes[1].plot(x_d, df['final_htse'], 's', color='green')
        axes[1].plot(x_d, fit_2 + htse_0, '--', color='purple')
        axes[1].text(0.95, 0.05, f'$R^2={r2_2:.2f}, p={p_2:.1e}$\n$H-H_0 \propto D^{{{s_2:.2f}}}$', ha='right', va='bottom', transform=axes[1].transAxes, bbox=dict(fc='wheat', alpha=0.5))
        axes[1].set_ylabel("Final H'_TSE")
        
        # --- [Plot 3] Component 2: Compression ---
        r2_3, p_3, d_rate_3, fit_3 = exp_decay_fit(x_d, df['reducible_hsie'])
        axes[2].set_title('Component 2: Compression (Exp. Decay)')
        axes[2].plot(x_d, df['final_hsie'], '^', color='orange')
        axes[2].plot(x_d, fit_3 + hsie_inf, '--', color='darkred')
        axes[2].text(0.95, 0.95, f'$R^2={r2_3:.2f}, p={p_3:.1e}$\n$H-H_\infty \propto e^{{-{d_rate_3:.1e}D}}$', ha='right', va='top', transform=axes[2].transAxes, bbox=dict(fc='wheat', alpha=0.5))
        axes[2].set_ylabel("Final H'_SIE")
        
        # --- [Plot 4] Internal Cost Trend ---
        htse_red_sq = np.maximum(0, df['reducible_htse'])**2
        hsie_red_sq = np.maximum(0, df['reducible_hsie'])**2
        df['L_ideal_reducible'] = np.sqrt(BASE_CONFIG['w1'] * htse_red_sq + BASE_CONFIG['w2'] * hsie_red_sq)
        axes[3].plot(x_d, df['L_ideal_reducible'], 'p', color='purple')
        axes[3].set_title('Internal Cost $\mathcal{L}_{ideal, red}$ Trend')
        axes[3].set_ylabel('Reducible Ideal Norm')

        # --- [Plot 5] The Core Law: Performance vs. Cost ---
        ax4 = axes[4]
        ax4.set_title('The Core Law: Performance vs. Cost', fontsize=16)
        r2_4, p_4, s_4, fit_4 = power_law_fit(df['L_ideal_reducible'], df['reducible_loss'])
        ax4.plot(df['L_ideal_reducible'], df['reducible_loss'], 'd', color='black')
        ax4.plot(df['L_ideal_reducible'], fit_4, '--', color='magenta', lw=2.5)
        core_text = (f'Core Predictive Law:\n' 
                     f'$R^2 = {r2_4:.4f}$ ($p={p_4:.1e}$)\n'
                     f'$L_{{red}} \propto \mathcal{{L}}_{{ideal, red}}^{{{s_4:.2f}}}$')
        ax4.text(0.95, 0.95, core_text, transform=ax4.transAxes, fontsize=12, va='top', ha='right', bbox=dict(fc='wheat', alpha=0.5))
        ax4.set_xlabel('Reducible Ideal Norm $\mathcal{L}_{ideal, red}$', fontsize=12)
        ax4.set_ylabel('Reducible Test Loss', fontsize=12)

        # --- Final Formatting ---
        for i in range(5):
            if i < 4:
                axes[i].set_xlabel('Dataset Size D')
            axes[i].set_xscale('log')
            axes[i].set_yscale('log')
            axes[i].grid(True, which='both', linestyle='--')
        
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        
        # --- Save results ---
        output_img_path = f"scaling_law_unified_theory_seed_{seed}_epochs_{BASE_CONFIG['epochs']}.png"
        output_csv_path = f"results_seed_{seed}_epochs_{BASE_CONFIG['epochs']}.csv"
        
        plt.savefig(output_img_path, dpi=150)
        df.to_csv(output_csv_path, index=False)
        
        print(f"Plot saved to: {output_img_path}")
        print(f"Experiment data saved to: {output_csv_path}")
        
        plt.show()
